In [25]:
import json
import random
from datasets import load_dataset

/Users/jdidio/kDrive/EPFL/Master - DS/MA4/MNLP/project-code-2024/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Open the JSON file
with open('../datasets/M1_preference_data_15052024.json', 'r') as f:
    data = json.load(f)

In [3]:
count = 0
for i in data:
    count += len(i['preference'])

count

26738

In [4]:
count = 0
for i in data:
    if "Options:" in i['question_complete']:
        count += 1

In [5]:
count

792

In [6]:
# Create a list to store individual preferences
preferences_train = []
preferences_test = []
# Extract preference items from the data
for question in data:
    size = len(question['preference'])
    test = random.randint(0, size - 1)
    for count, preference_item in enumerate(question['preference']):
        if count == test:
            if preference_item['overall'] == 'A':
                preferences_test.append({
                    'prompt': question['question_complete'],
                    'chosen': preference_item['A'],
                    'rejected': preference_item['B'],
                })
            elif preference_item['overall'] == 'B':
                preferences_test.append({
                    'prompt': question['question_complete'],
                    'chosen': preference_item['B'],
                    'rejected': preference_item['A'],
                })
            else:
                print(preference_item['overall'])

        else:
            if preference_item['overall'] == 'A':
                preferences_train.append({
                    'prompt': question['question_complete'],
                    'chosen': preference_item['A'],
                    'rejected': preference_item['B'],
                })
            elif preference_item['overall'] == 'B':
                preferences_train.append({
                    'prompt': question['question_complete'],
                    'chosen': preference_item['B'],
                    'rejected': preference_item['A'],
                })
            else:
                print(preference_item['overall'])

In [7]:
len(preferences_train)

25216

In [8]:
len(preferences_test)

1522

In [9]:
# Write the preferences to a jsonl file
with open('../datasets/class_train.jsonl', 'w') as file:
    for preference in preferences_train:
        file.write(json.dumps(preference) + '\n')

In [10]:
# Write the preferences to a jsonl file
with open('../datasets/class_test.jsonl', 'w') as file:
    for preference in preferences_test:
        file.write(json.dumps(preference) + '\n')

## SFT dataset

In [43]:
train = load_dataset('json', data_files='../datasets/class_train.jsonl')

In [44]:
def transform_to_sft(examples):
    return {"question": examples["prompt"], "answer": examples["chosen"]}

In [45]:
train = train.map(transform_to_sft, batched=True, remove_columns=["prompt", "chosen", "rejected"]).shuffle(seed=42)
split = train['train'].train_test_split(test_size=0.1)
split.save_to_disk('../datasets/class_sft_datasetdict')

Saving the dataset (1/1 shards): 100%|██████████| 2522/2522 [00:00<00:00, 317668.24 examples/s]
